# Settings

In [2]:
# API_KEY = "sk-proj-***EQsJNM7EjjiVoCGkj7szMq4Jlxx4x_sfQlEA"

In [4]:
import os
os.environ['OPENAI_API_KEY'] = API_KEY

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5
)

llm.invoke("hola")

AIMessage(content='¡Hola! ¿Cómo puedo ayudarte hoy?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpsxsF5VZEAGrUHCgO6H1a3HkQqGI', 'finish_reason': 'stop', 'logprobs': None}, id='run-87aec746-f517-4b8c-9230-a4aef3e4aa84-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, BaseMessage

chat_history = [
    HumanMessage(content="cual es la capitald e FRancia?"),
    AIMessage(content="Es paris")
]
chat_history

[HumanMessage(content='cual es la capitald e FRancia?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Es paris', additional_kwargs={}, response_metadata={})]

# State

In [107]:
from typing import TypedDict, Annotated, Literal
from pydantic import BaseModel
from langgraph.graph.message import add_messages

class AgentState(TypedDict):  # BaseModel
    messages: Annotated[list[BaseMessage], add_messages]
    decision: Literal['ok', 'fin']

# Tools

In [108]:
from langchain.tools import tool
from langgraph.prebuilt import ToolNode
import requests

In [109]:
@tool
def send_telegram_message(mensaje: str) -> str:
    """
    Envía un mensaje de texto en un chat de Telegram. Útil si quiero abrir un siniestro.
    
    Args:
        mensaje (str): El contenido del mensaje.
    Returns:
        str: Mensaje de confirmación.
    """
    
    url_getIdChat = 'https://api.telegram.org/bot'+bot_token+'/getUpdates'
    response = requests.get(url_getIdChat)

    bot_message = mensaje
    url_sendMessage = 'https://api.telegram.org/bot'+bot_token+'/sendMessage?chat_id='+bot_chatId+'&parse_mode=Markdown&text='+bot_message

    response = requests.get(url_sendMessage)
    return response.text

# Prompts

In [110]:
from langchain.prompts import ChatPromptTemplate

PROMPT_SUPERVISOR = ChatPromptTemplate.from_template(
"""Un CLIENTE está haciendo consultas a una compañía de Seguros.
    
    Tu objetivo es determinar si el CLIENTE está hablando de temas de seguros de hogar \
    o bien si empieza a hablar de cosas que no tienen que ver con seguros, siniestros de hogar, \
    coberturas de seguros o similar.
    
    Tienes que clasificar la última de sus consultas: 
    - Responde de forma escueta "ok" si la conversación es apropiada a \
    una conversación de seguros.
    - Responde de forma escueta "fin" si el CLIENTE consultar cosas que \
    no tienen que ver con seguros.
    
    Aquí tienes la conversación :
    --------------------------- 
    {messages}
    ---------------------------

    Tu respuesta (ok/fin) :  
    """
)

PROMPT_ASESOR = ChatPromptTemplate.from_template(
    """Un CLIENTE que está haciendo consultas a una compañía de Seguros.
    
    Tu objetivo es proporcionarle una respuesta en función del conocimiento de las coberturas.
    
    Instrucciones:
    - Si la pregunta no tiene que ver con coberturas, respóndele como sepas hacerlo. \
    Pero no te enredes demasiado.
    - Si ves que tiene un problema de verdad, le preguntas si quiere abrir un \
    ticket de Siniestro. 
    - Si quiere abrir un siniestro, envías un Telegram cuando sepas lo que le ha ocurrido con \
    exactitud y sepas también su email. Cuando no sepas alguna de estas cosas, le preguntas.
    
    En cualquier caso, se amable y muy educado.
    
    Aquí tienes la conversación :
    --------------------------- 
    {messages}
    ---------------------------
    """
)


# Nodes


In [121]:
def supervisor_action(state: AgentState):
    
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0
    )
    chain = PROMPT_SUPERVISOR | llm
    response = chain.invoke({"messages": state['messages']})
    decision = response.content
#     print(f">>> Supervisor: {decision}")
    return {
        "decision": decision
    }

def asesor_action(state: AgentState):
    
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.5
    )
    llm = llm.bind_tools([send_telegram_message])
    chain = PROMPT_ASESOR | llm
    response = chain.invoke({"messages": state['messages']})
    answer = response.content
    print(f">>> Asesor: {answer}")
    return {
        "messages": [response]
    }

# Routing logic

In [122]:
def supervisor_decision(state: AgentState):
    if state['decision'] == 'ok':
        return "Asesor"
    else:
        return END
    
def asesor_decision(state: AgentState):
    messages = state.get("messages", [])
    ai_message = messages[-1]
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls)>0:
        return "tools"
    else:
        return END

# Graph

In [123]:
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

def update_graph():
    
    graph = StateGraph(AgentState)
    
    tools_node = ToolNode([send_telegram_message])
    
    # nodos
    graph.add_node("Supervisor", supervisor_action)
    graph.add_node("Asesor", asesor_action)
    graph.add_node("tools", tools_node)
    
    # edges
    graph.add_edge(START, "Supervisor")
    graph.add_conditional_edges("Supervisor", supervisor_decision)
#     graph.add_edge("Asesor", END)
    graph.add_conditional_edges("Asesor", asesor_decision)
    graph.add_edge("tools", "Asesor")
    
    memory = MemorySaver()
    return graph.compile(checkpointer=memory)

# Simulación de la conversación

In [124]:
# hola se me ha roto el microondas, me lo cubre el seguro?
# si, me gustaría abrir un ticket de siniestro
# meti una cuchara en el microondas y explot todo. mi correo electroico es efc@ucm.es

graph = update_graph()

config = {"configurable": {"thread_id": "1"}}

user_input = input("\nPregunta del usuario: ('stop' para parar)")
while user_input != 'stop':
    messages = graph.invoke(
        input={"messages": [HumanMessage(content=f"{user_input}")]},
        config=config
    )
    user_input = input("\nPregunta del usuario: ('stop' para parar)")


Pregunta del usuario: ('stop' para parar)# hola se me ha roto el microondas, me lo cubre el seguro?
>>> Asesor: Hola, gracias por tu consulta. Lamentablemente, los daños a electrodomésticos como el microondas suelen no estar cubiertos por los seguros de hogar estándar, a menos que se trate de un daño específico cubierto en tu póliza. 

Si el daño fue causado por un evento cubierto, como un incendio o una inundación, podrías tener derecho a una reclamación. ¿Te gustaría abrir un ticket de siniestro para que podamos revisarlo? Si es así, necesitaría saber más detalles sobre lo ocurrido y tu dirección de correo electrónico.

Pregunta del usuario: ('stop' para parar)# si, me gustaría abrir un ticket de siniestro
>>> Asesor: ¡Perfecto! Para poder abrir el ticket de siniestro, necesito que me proporciones más detalles sobre lo ocurrido con el microondas. Además, por favor indícame tu dirección de correo electrónico para que podamos contactarte.

Pregunta del usuario: ('stop' para parar)# me

In [130]:
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

# hola se me ha roto el microondas, me lo cubre el seguro?
================================== Ai Message ==================================

Hola, gracias por tu consulta. Lamentablemente, los daños a electrodomésticos como el microondas suelen no estar cubiertos por los seguros de hogar estándar, a menos que se trate de un daño específico cubierto en tu póliza. 

Si el daño fue causado por un evento cubierto, como un incendio o una inundación, podrías tener derecho a una reclamación. ¿Te gustaría abrir un ticket de siniestro para que podamos revisarlo? Si es así, necesitaría saber más detalles sobre lo ocurrido y tu dirección de correo electrónico.
================================ Human Message =================================

# si, me gustaría abrir un ticket de siniestro
================================== Ai Message ==================================

¡Perfecto! Para poder abrir el ticket de siniestr